In [63]:
# Importing necessary libraries and modules
import pandas as pd
import random
from rdflib.namespace import RDF, RDFS, FOAF, XSD, URIRef
from rdflib import Graph
import pandas as pd
from rdflib import Namespace
from rdflib import Literal

In [64]:
# Read the CSV files
authors1 = pd.read_csv('authors1.csv')
authorsPapers = pd.read_csv('authorsPapers.csv')
chair = pd.read_csv('chair.csv')
conference = pd.read_csv('conference.csv')
conferenceProceedings = pd.read_csv('conferenceProceedings.csv')
editor = pd.read_csv('editor.csv')
journal = pd.read_csv('journal.csv')
journal_volumes = pd.read_csv('journal_volumes.csv')
papers = pd.read_csv('papers.csv')
proceeding = pd.read_csv('proceeding.csv')
publications = pd.read_csv('publications.csv')
reviewers = pd.read_csv('reviewers.csv')
reviews = pd.read_csv('reviews.csv')
supervisors = pd.read_csv('supervisors.csv')
topic = pd.read_csv('topic.csv')
topicsPapers = pd.read_csv('topicsPapers.csv')
topicsProceedingsVolumes = pd.read_csv('topicsProceedingsVolumes.csv')
volumes = pd.read_csv('volumes.csv')


In [65]:
# Creating a graph
g = Graph()

In [66]:
# Create many URIRefs in the same namespace, i.e. URIs with the same prefix
LAB = Namespace("http://SDM_LAB2.org/")

In [67]:
# Bind the lab namespace to a prefix for more readable output
g.bind('lab',LAB)

Author [writes] -> Paper

In [68]:
authorsPapers.head(2)

,authorID,authorRef,Name,paperID,ID
0,a0,57285646800,Zhu F.,p361,b131ac160af2c3ef91aff47f6578067183ca4c4b
1,a1,58827616000,Ding J.,p361,b131ac160af2c3ef91aff47f6578067183ca4c4b


In [69]:
for k in range(len(authorsPapers['authorID'])):
    g.add((URIRef(LAB+authorsPapers['authorID'][k]), RDF.type, LAB.Author))

Paper [publishedAs] -> Publication

In [70]:
publications.head(2)

,publicationID,paperID,title,abstract
0,pub0,p0,"Reproducible, interactive, scalable and extens...",NaN
1,pub1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ..."


In [71]:
for k in range(len(publications['paperID'])):
    g.add((URIRef(LAB+publications['paperID'][k]), RDF.type, LAB.Paper))
    g.add((URIRef(LAB+publications['paperID'][k]), RDF.type, LAB.Publication))

Paper [paperRelatedTo] -> Topic

In [72]:
topicsPapers.head(2)

,topicID,topicName,paperID,ID,title,abstract,confjorID,confjorTitle
0,t7270,Embedding strategies,p0,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,"Reproducible, interactive, scalable and extens...",NaN,j6283,Chemical Engineering Journal
1,t860,Chlorine compounds,p1,fa5853fdef7d2f6bb68203d187ddacbbddc63a8b,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",j15687,Global Change Biology


In [73]:
for k in range(len(topicsPapers['paperID'])):
    g.add((URIRef(topicsPapers['topicID'][k]), RDF.type, LAB.Topic))

Conference [isIn] -> Proceedings

In [74]:
conferenceProceedings.head(2)

,conferenceID,conferenceRef,name,year,edition,conProIds,proceedingTitle
0,c0,197283.0,11th International Conference on Hard and Elec...,2023.0,1.0,cp0,proceeding0
1,c1,175805.0,"18th International Conference on Tangible, Emb...",2024.0,1.0,cp1,proceeding1


In [75]:
for k in range(len(conferenceProceedings['conferenceID'])):
    g.add((URIRef(LAB+conferenceProceedings['conProIds'][k]), RDF.type, LAB.Proceedings))

Journal [isOf] -> Volume

In [76]:
journal_volumes.head(2)

,journalID,journalRef,journal,year,volumeName,journalVID
0,j0,2-s2.0-85182825734,Health Information Science and Systems,2024,journal0,jv0
1,j1,2-s2.0-85184294976,Humanities and Social Sciences Communications,2024,journal1,jv1


In [77]:
for k in range(len(journal_volumes['journalID'])):
    g.add((URIRef(LAB+journal_volumes['journalVID'][k]), RDF.type, LAB.Volume))
    g.add((URIRef(LAB+journal_volumes['journalID'][k]), RDF.type, LAB.Journal))

Paper [isPublishedInConference] -> Proceedings and Paper [isPublishedInJournal] -> Volume

In [78]:
papers.head(2)

,paperID,ID,title,abstract,confjorID,confjorTitle,proceedingsVolumesIds
0,p0,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,"Reproducible, interactive, scalable and extens...",NaN,j6283,Chemical Engineering Journal,jv15795
1,p1,fa5853fdef7d2f6bb68203d187ddacbbddc63a8b,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",j15687,Global Change Biology,cp1


In [79]:
for k in range(len(papers['paperID'])):
    if papers['confjorID'][k][0] == 'c':
        g.add((URIRef(LAB+papers['proceedingsVolumesIds'][k]), RDF.type, LAB.Proceedings))
    else:
        g.add((URIRef(LAB+papers['proceedingsVolumesIds'][k]), RDF.type, LAB.Volume))

Proceedings [proceedingsRelatedTo] -> Topic and Volumes [volumeRelatedTo] -> Topic

In [80]:
topicsProceedingsVolumes.head(2)

,topicID,jcIDs,vpIDs
0,t7270,j15795,jv15795
1,t860,c1,cp1


In [81]:
for k in range(len(topicsProceedingsVolumes['vpIDs'])):
    if topicsProceedingsVolumes['vpIDs'][k][0] == 'c':
        g.add((URIRef(LAB+topicsProceedingsVolumes['vpIDs'][k]), RDF.type, LAB.Proceedings))
    else:
        g.add((URIRef(LAB+topicsProceedingsVolumes['vpIDs'][k]), RDF.type, LAB.Volume))

Creating Chair [handleConference] -> Conference and Editor [handleJournal] -> [Journal]

In [82]:
chair.head(2)

,conferenceID,authorID,chairID
0,c39,a57479,chair0
1,c170,a57840,chair1


In [83]:
editor.head(2)

,journalID,authorID,editorID
0,j11456,a57479,editor0
1,j16528,a57840,editor1


In [84]:
for k in range(len(chair['conferenceID'])):
    g.add((URIRef(LAB+chair['authorID'][k]), RDF.type, LAB.Chair))
    g.add((URIRef(LAB+chair['conferenceID'][k]), RDF.type, LAB.Conference))

In [85]:
for k in range(len(editor['journalID'])):
    g.add((URIRef(LAB+editor['authorID'][k]), RDF.type, LAB.Editor))
    g.add((URIRef(LAB+editor['journalID'][k]), RDF.type, LAB.Journal))

Creating Supervisor [assigns] -> Reviewer

In [86]:
supervisors.head(2)

,authorID,supervisorID,confjorID,supervisorsIDs
0,a38993,s38993,j6283,editor0
1,a66571,s66571,j15687,editor1


In [87]:
for k in range(len(supervisors['supervisorsIDs'])):
    g.add((URIRef(LAB+supervisors['authorID'][k]), RDF.type, LAB.Reviewer))

Creating Reviewer [reviewed] -> Review

In [88]:
reviewers.head(2)

,authorRef,Name,reviewID,publicationID,paperID,title,reviewDecision,reviewDecisionBoolean,authorID
0,57989947600,Yang K.,r362,pub212,p212,Lecture Notes in Computer Science (including s...,rejected,0,a15795
1,57555731100,Liu J.,r73,pub324,p324,Fuzzy Graph Theory,accepted,1,a860


In [89]:
for k in range(len(reviewers['reviewID'])):
    g.add((URIRef(LAB+reviewers['reviewID'][k]), RDF.type, LAB.Review))

Creating Review [isReviewOf] -> Paper

In [90]:
reviews.head(2)

,reviewID,publicationID,paperID,title,reviewDecision,reviewDecisionBoolean
0,r0,pub0,p0,"Reproducible, interactive, scalable and extens...",rejected,0
1,r1,pub1,p1,High-Dimensional Probability: An Introduction ...,accepted,1


In [91]:
for k in range(len(reviews['reviewID'])):
    g.add((URIRef(LAB+reviews['paperID'][k]), RDF.type, LAB.Paper))

In [92]:
g.serialize('data/tbox_abox_connection.ttl',format = 'ttl')

<Graph identifier=N3bccf3ade42742ce9babf218c140e100 (<class 'rdflib.graph.Graph'>)>